In [1]:
# Install WhiteboxTools (only needed once)
# !pip install whitebox

from whitebox import WhiteboxTools

# --------------------------
# Configuration
# --------------------------
WORKDIR = "/Users/shg096/Desktop/RiverLakeNetwork/examples/Plots"
DEM = "/Users/shg096/Downloads/elv_n30w120/n50w110_elv.tif"
STREAM_THRESHOLD = 10000  # flow accumulation threshold (cells)

# --------------------------
# Initialize WhiteboxTools
# --------------------------
wbt = WhiteboxTools()
wbt.work_dir = WORKDIR
wbt.verbose = True

# --------------------------
# 1. Fill depressions
# --------------------------
wbt.fill_depressions_wang_and_liu(
    dem=DEM,
    output="dem_filled.tif",
    fix_flats=False,
)

# --------------------------
# 2. Flow direction (D8)
# --------------------------
wbt.d8_pointer(
    dem="dem_filled.tif",
    output="flow_dir.tif"
)

# --------------------------
# 3. Flow accumulation
# --------------------------
wbt.d8_flow_accumulation(
    "dem_filled.tif",
    output="flow_acc.tif",
    out_type="cells"
)

# --------------------------
# 4. Extract streams
# --------------------------
wbt.extract_streams(
    flow_accum="flow_acc.tif",
    output="streams.tif",
    threshold=STREAM_THRESHOLD
)

# --------------------------
# 5. Convert streams raster to vector (river shapefile)
# --------------------------
wbt.raster_streams_to_vector(
    streams="streams.tif",
    d8_pntr="flow_dir.tif",
    output="rivers.shp"
)

# --------------------------
# 6. Delineate subbasins
# --------------------------
wbt.subbasins(
    d8_pntr="flow_dir.tif",
    streams="streams.tif",
    output="subbasins.tif"
)

# Convert subbasins to polygons
wbt.raster_to_vector_polygons(
    i="subbasins.tif",
    output="basins.shp"
)

print("DEM → rivers → basins completed successfully!")



./whitebox_tools --run="FillDepressionsWangAndLiu" --wd="/Users/shg096/Desktop/RiverLakeNetwork/examples/Plots" --dem='/Users/shg096/Downloads/elv_n30w120/n50w110_elv.tif' --output='dem_filled.tif' -v --compress_rasters=False

****************************************
* Welcome to FillDepressionsWangAndLiu *
* Powered by WhiteboxTools             *
* www.whiteboxgeo.com                  *
****************************************
Reading data...
progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 33%
Progress: 34%
Progress: 35%
Progress: 36%
Progress: 37%
Progress: 38%
Progress: 39%
Pr

In [2]:
import geopandas as gpd
from shapely.geometry import Point
import networkx as nx

# Load the river shapefile
riv = gpd.read_file("rivers.shp")
riv = riv.sort_index().reset_index(drop=True)

# Assign unique IDs
riv["COMID"] = riv.index + 1

# Compute segment length
riv["Length"] = riv.geometry.length

# Build networkx graph based on start/end points
G = nx.DiGraph()
for idx, row in riv.iterrows():
    start_pt = row.geometry.coords[0]
    end_pt = row.geometry.coords[-1]
    G.add_node(row["COMID"], geom=row.geometry, end=end_pt)
    
# Find downstream for each segment (matching end point to start point of another segment)
next_down = []
for idx, row in riv.iterrows():
    downstream_id = None
    row_end = row.geometry.coords[-1]
    for jdx, r in riv.iterrows():
        if r.geometry.coords[0] == row_end:
            downstream_id = r["COMID"]
            break
    if downstream_id is None:
        downstream_id = -9999  # outlet
    next_down.append(downstream_id)

riv["NextDownCOMID"] = next_down

# Save the new river network shapefile
riv.to_file("rivers_network.shp")

print("River network shapefile with COMID, NextDownCOMID, and Length created!")


KeyboardInterrupt: 

In [3]:
wbt.fill_depressions_wang_and_liu?

Signature:
wbt.fill_depressions_wang_and_liu(
    dem,
    output,
    fix_flats=True,
    flat_increment=None,
    callback=None,
)
Docstring:
Fills all of the depressions in a DEM using the Wang and Liu (2006) method. Depression breaching should be preferred in most cases.

Keyword arguments:

dem -- Input raster DEM file.
output -- Output raster file.
fix_flats -- Optional flag indicating whether flat areas should have a small gradient applied.
flat_increment -- Optional elevation increment applied to flat areas.
callback -- Custom function for handling tool text outputs.
File:      ~/Desktop/RiverLakeNetwork/env/RiverLakeEnv/lib/python3.9/site-packages/whitebox/whitebox_tools.py
Type:      method